<a href="https://colab.research.google.com/github/manavmarya/Train-YOLO/blob/main/train_yolo_algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import numpy as nm
import pandas as pd
import glob
import os
import sys 
import shutil
from shutil import copyfile

def csv_to_df(filename_url):
    df = pd.read_csv(filename_url)
    return df

def format_df(annotation_df, filesize):
    '''Converting xyxy to xywh'''
    annotation_df["x1"] = annotation_df["x1"]/filesize[0] 
    annotation_df["y1"] = annotation_df["y1"]/filesize[1]
    annotation_df["w"] = abs(annotation_df["x2"]-annotation_df["x1"])/filesize[0]
    annotation_df["h"] = abs(annotation_df["y2"]-annotation_df["y1"])/filesize[1]
    cols = ["x1", "y1", "w", "h"]
    annotation_df[cols] = annotation_df[cols].clip(upper=0.999)
    return annotation_df

def split_train_valid(split_pct):
    '''Splitting training and validation datasets based on split_pct (Default is 90:10)'''
    current_dir = "./data/images"
    file_train = open("data/train.txt", "w")  
    file_val = open("data/val.txt", "w")  
    counter = 1  
    index_test = round(100 / split_pct)  
    for fullpath in glob.glob(current_dir + '/*.jpg'):  
        title, ext = os.path.splitext(os.path.basename(fullpath))
        if counter == index_test:
            counter = 1
            file_val.write("./images" + "/" + title + ".jpg" + "\n")
        else:
            file_train.write("./images" + "/" + title + ".jpg" + "\n")
        counter = counter + 1
    file_train.close()
    file_val.close()

def create_labels(df):
    current_dir = "./data/labels"
    for i in df.index:
        curr_filename = df["image_name"][i]
        if i == 0 or curr_filename != prev_filename:
            if i != 0:
                label_file.close()
            label_file = open("data/labels/" + df["image_name"][i].split(".")[0] + ".txt", "w")
        label_file.write("0 " + str(df["x1"][i]) + " " + str(df["y1"][i]) + " " + str(df["w"][i]) + " " + str(df["h"][i]) + "\n")
        prev_filename = curr_filename
        

annotation_df = csv_to_df("WebMarket_coco_gt_object.csv")
annotation_df = format_df(annotation_df, [2272,1704])
split_train_valid(10)
create_labels(annotation_df)



Using YOLO git (already done, hence commented)

In [11]:
#!git clone  'https://github.com/ultralytics/yolov5.git'
#!pip install -qr './yolov5/requirements.txt'

     |████████████████████████████████| 645kB 7.8MB/s 


In [ ]:
!echo -e 'train: /content/data/train.txt\nval: /content/data/val.txt\n\nnc: 1\nnames: ['Objects']' >> supermarket.yaml
!cat 'supermarket.yaml'
shutil.copyfile('/content/supermarket.yaml', '/content/yolov5/supermarket.yaml')
!sed -i 's/nc: 80/nc: 1/g' ./yolov5/models/yolov5s.yaml
!python yolov5/train.py --img 640 --batch 8 --epochs 10 --data supermarket.yaml --cfg models/yolov5s.yaml --name Supermarket

In [ ]:
!python yolov5/detect.py --source /content/data/valid/ --weights '/content/runs/train/Supermarket4/weights/best.pt' 